# Redis Pub-Sub

In [1]:
import redis
import inspect
import os
import uuid

In [2]:
client = redis.Redis(host=os.environ.get('REDIS_HOST'), port=os.environ.get('REDIS_PORT'), 
        decode_responses=True)
client.ping()

In [47]:
class RedisSSE:
    REDIS_HOST = os.environ.get('REDIS_HOST')
    REDIS_PORT = os.environ.get('REDIS_PORT')

    def __init__(self, pattern) -> None:
        assert bool(pattern)
        assert isinstance(pattern, str)
        self.pattern = pattern
        self.redis_conn = redis.Redis(host=self.REDIS_HOST, port=self.REDIS_PORT)
        self.redis_conn.config_set('notify-keyspace-events', 'Kh')
        self.redis_pubsub = self.redis_conn.pubsub(ignore_subscribe_messages=True)
        self.db = 0
        self.keyspace_pattern = f"__keyspace@{self.db}__:"
        self.redis_pubsub.psubscribe(f"{self.keyspace_pattern}{self.pattern}")

    def listen(self):
        files = {}
        processing_states = {'queued', 'started'}
        yield "data: connected and listening for events\n\n"
        for message in self.redis_pubsub.listen():
            if message:
                print(message)

In [48]:
pattern = f"rq:job:mindkosh-backend-test-3/{uuid.uuid4()}/files/test-dataset/{uuid.uuid4()}"

In [49]:
pattern

'rq:job:mindkosh-backend-test-3/42cfd844-1a97-4a01-9aaf-4ddfbb681d5f/files/test-dataset/f7087380-4b4d-4e14-a92a-3010832d7991'

In [50]:
redis_sse = RedisSSE(pattern=f"{pattern}/*")

In [51]:
sse_generator = redis_sse.listen()
while True:
    print(next(sse_generator))

data: connected and listening for events


{'type': 'pmessage', 'pattern': b'__keyspace@0__:rq:job:mindkosh-backend-test-3/42cfd844-1a97-4a01-9aaf-4ddfbb681d5f/files/test-dataset/f7087380-4b4d-4e14-a92a-3010832d7991/*', 'channel': b'__keyspace@0__:rq:job:mindkosh-backend-test-3/42cfd844-1a97-4a01-9aaf-4ddfbb681d5f/files/test-dataset/f7087380-4b4d-4e14-a92a-3010832d7991/fba3caca-133c-4090-80fa-f69964b39849', 'data': b'hset'}


KeyboardInterrupt: 

In [63]:
sse_generator = redis_sse.listen()

In [ ]:
next(sse_generator)

{'type': 'pmessage', 'pattern': b'__keyspace@0__:rq:job:mindkosh-backend-test-3/42cfd844-1a97-4a01-9aaf-4ddfbb681d5f/files/test-dataset/f7087380-4b4d-4e14-a92a-3010832d7991/*', 'channel': b'__keyspace@0__:rq:job:mindkosh-backend-test-3/42cfd844-1a97-4a01-9aaf-4ddfbb681d5f/files/test-dataset/f7087380-4b4d-4e14-a92a-3010832d7991/cabd7428-4415-49b7-bc27-0749b120b88c', 'data': b'hset'}
